# 기본옵션

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
pip install selenium

     |████████████████████████████████| 911kB 2.7MB/s 


In [0]:
pip install KoNLPy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 2.9MB 36.0MB/s 
     |████████████████████████████████| 92kB 7.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [0]:
import pandas as pd 
import numpy as np
import re
import os
import itertools
import json
from collections import OrderedDict

import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt 

import warnings
warnings.filterwarnings("ignore")

# 한국어 전처리
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import json
import sys

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras_preprocessing import sequence

from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

from sklearn.metrics import accuracy_score

# 모델링
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv1D

from keras import metrics
from keras import optimizers
from keras.layers import MaxPooling2D
from keras.layers import Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense

import gensim


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


# Data Krawling

In [0]:
import pandas as pd
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.max.colwidth',60)
pd.options.display.max_columns = 100

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
from bs4 import BeautifulSoup
import re

## url list

In [0]:
# 원하는 유튜버 '동영상'에 들어가기 
driver = webdriver.Chrome('/content/drive/My Drive/Colab Notebooks/chromedriver.exe')
driver.maximize_window()
driver.get('https://www.youtube.com/channel/UCtj_00nn_viF6wJNeqgpEiA/videos') # ★★★ 변경
body = driver.find_element_by_tag_name('body')
time.sleep(1.5)#브라우저 로딩시간기다려야함

# 유튜브는 스크롤해야 전체를 볼 수 있음 # 전체 영상 개수에 따라 조절해야함
endk = 110 # 5번 내림 # 500개면 한 100번정도 넉넉히 내리기 # 시험해보려고 1번만 내림 # ★★★ 변경
while endk:
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(0.3)
    endk -= 1    
start_page = driver.page_source 
soup = BeautifulSoup(start_page,'lxml') # 전체 영상 목록의 page-source 이니 soup으로

# 각 영상 페이지의 url 모음
video_list = soup.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

channel_url = [] # ★★★ 이름 변경
base_url = 'http://www.youtube.com'
for i in range(len(video_list)):
    url = base_url+video_list[i].find('a',{'id':'thumbnail'})['href']
    channel_url.append(url) # ★★★ 이름 변경
    

WebDriverException: ignored

In [0]:
len(youtube_url)

## 동영상 별 데이터수집
- 추출한 모든 영상의 url을 입력하여, 각 페이지 안으로 들어가 정보를 수집
- 채널명/ 구독자 수 / 썸네일 / 제목 / 업로드 날짜 / 조회수 / 좋아요 / 싫어요 / 영상길이 / 카테고리

In [0]:
# ★★★ 이름 변경
channel = pd.DataFrame({'channel_name' :[],
                           'subscribers' : [],
                           'thumbnail' :[],
                           'video_name':[],
                           'video_length':[],
                           'upload_date':[],
                           'hits':[],
                           'likes_num':[],
                           'dislikes_num':[],
                           'category':[] })

for i in range(0,len(channel_url)):# ★★★ 이름 변경
    
    try :# 오휴생겨서 try-except 추가        
        channel_name = soup.find('title').text
        subscribers = soup.find('yt-formatted-string',{'id':'subscriber-count'}).text
        thumbnail = video_list[i].find('img', {'src' : True})['src'] 
        video_name = video_list[i].find('a',{'id':'video-title'}).text
        video_length = video_list[i].find('span', {'class':'style-scope ytd-thumbnail-overlay-time-status-renderer'}).text.split('\n')[1]

        # 각 동영상 페이지 들어가기 시작 
        start_url = channel_url[i] # ★★★ 이름 변경
        driver.get(start_url) 
        body = driver.find_element_by_tag_name('body')

        time.sleep(1.5)

        # 댓글까지 쭉 보려면 스크롤 # 사실상 지금 댓글은 안보니 1번만 내리기
        endk = 1 # 5번 내림
        while endk:
            driver.find_element_by_tag_name('body').send_keys(Keys.END)
            time.sleep(0.3) # 내리는 시간 간격 
            endk -= 1    
        page = driver.page_source 
        html = BeautifulSoup(page,'lxml') # 각 페이지의 page_source 이니 html로


        # 각 동영상 페이지 들어가서 얻을 수 있는 정보
        pre_upload_date = html.find('div',{'id':'date'})
        upload_date = pre_upload_date.find('yt-formatted-string', {'class':'style-scope ytd-video-primary-info-renderer'}).text  
        pre_hits = html.find('yt-view-count-renderer', {'class':'style-scope ytd-video-primary-info-renderer'})
        hits = pre_hits.find('span',{'class':'view-count style-scope yt-view-count-renderer'}).text.split()[1]
        likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'
        dislikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'
        category = 'sports' # 바꾸면됨


        # 마지막에 모은 정보 합치기
        insert_data = pd.DataFrame({'channel_name' :[channel_name],
                                   'subscribers' : [subscribers],
                                   'thumbnail' :[thumbnail],
                                   'video_name':[video_name],
                                   'video_length':[video_length],
                                   'upload_date':[upload_date],
                                   'hits' :[hits],
                                   'likes_num':[likes_num],
                                   'dislikes_num':[dislikes_num],
                                   'category':[category]})
    
        channel = channel.append(insert_data) # ★★★ 이름 변경
    
    except KeyError as e: # error뜬거는 포함 안하고 저장됨 (error 뜨는거 찾을 때 마다 그냥 넣어버리기)
        print(k, i)
    except AttributeError as a: 
        print(a ,i)
    except IndexError as e:
        print(e,i)
    except TypeError as t:
        print(t, i)

In [0]:
len(channel)
channel.head() # 이건 걍 무시해두됨 이미 저장하고 한번 더 돌리고 그냥 꺼버려서 그럼

In [0]:
channel.to_csv('channel.csv',encoding='utf-8-sig')

# Text Feature Extraction

## 전처리

### 텍스트 정규화 (노이즈 제거)

In [0]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/small_total_train_set.csv", encoding='utf-8')

In [0]:
# 불용어사전 읽어오기
f = open("/content/drive/My Drive/Colab Notebooks/ko/stopwords.txt", 'r', encoding='utf-8')
stopwords = []
for line in f.readlines():
    stopwords.append(line.rstrip())
f.close()

# #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(han,'',df['video_name'].iloc[i])
    clean_name_lst.append(a)
clean_name_lst=[]
for i in range(len(df)):
    a = re.sub(emoji_pattern,'',df['video_name'].iloc[i])
    b = re.sub(han,'',a)
    
    clean_name_lst.append(b)
    
df['clean_name']=clean_name_lst


In [0]:
from konlpy.tag import Okt
okt = Okt()

clean_word = [] 
for i in df['clean_name']:
    text = re.sub("[12?~3.,()''->\n]", '', i)
    word_text = okt.nouns(text)
    if True: word_text = [token for token in word_text if not token in stopwords]
        
    clean_word.append(' '.join(word_text))
    
df['clean_name'] = clean_word
df['okt_tok'] = df['clean_name'].apply(lambda x : okt.nouns(x))

okt_data = df[['category_id','video_name', 'channel_name', 'okt_tok']]

### 토큰화 및 단어집합

In [0]:
# 토크나이저 사용
# keras의 tokenizer는 default값이 띄어쓰기 구분이므로 okt로 토큰화된 단어들만 뽑아오는게 맞음
okt_vocab = Tokenizer()
okt_vocab.fit_on_texts(okt_data['okt_tok'])

In [0]:
threshold = 3
total_cnt = len(okt_vocab.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in okt_vocab.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 13024
등장 빈도가 2번 이하인 희귀 단어의 수: 7888
단어 집합에서 희귀 단어의 비율: 60.56511056511057
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 11.032626606613205


In [0]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
okt_vocab_size = total_cnt - rare_cnt 
print('단어 집합의 크기 :', okt_vocab_size)

단어 집합의 크기 : 5136


In [0]:
# 빈도수가 3미만인 단어를 w라고 저장
okt_freq = [w for w,c in okt_vocab.word_counts.items() if c < 3]

for w in okt_freq:
    del okt_vocab.word_index[w]
    del okt_vocab.word_counts[w]

okt_vocab_list = list(okt_vocab.word_index.keys())

In [0]:
okt_all = okt_data['okt_tok'].tolist()

titles = list(okt_data.video_name)
id_num = list(okt_data.category_id)

### 리스트 빈값 지우기

In [0]:

nan_lst =[]
for i in range(len(okt_all)):
  if not okt_all[i]:
    nan_lst.append(i)

nnnn = np.arange(len(okt_all)) # 이게 kkma_all의 인덱스
tttt = set(nnnn).difference(set(nan_lst)) # 인덱스에서 nan_lst뺀거!

okt_all = [okt_all[i] for i in tttt]
id_num = [id_num[i] for i in tttt]
titles = [titles[i] for i in tttt]

## 단어 임베딩

In [0]:
# sg=0 (cbow), sg=1 (skip-gram)
from gensim.models import Word2Vec
w2v = Word2Vec(sentences = okt_all, size = 100, window = 2, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

In [0]:
from gensim.models import FastText
fxt = FastText(sentences = okt_all, size = 100, window = 4, min_count = 1, workers = 4,sample=1e-3, iter=5, sg = 1)

## 문장 특성 추출 (단어 벡터의 평균값)

In [0]:
def get_features(words, model, num_features):
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [0]:
def get_dataset(data, model, num_features):
    dataset = list()
    
    for s in data:
        dataset.append(get_features(s, model, num_features))
        
    reviewFeatureVecs = np.stack(dataset)
    return reviewFeatureVecs

In [0]:
okt_w2v = get_dataset(okt_all, w2v, 100)

In [0]:
okt_fxt = get_dataset(okt_all, fxt, 100)

In [0]:
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/okt_w2v", okt_w2v) # save.npy
np.save("/content/drive/My Drive/딥러닝프로젝트_유튜브/데이터/단어임베딩/okt_fxt", okt_fxt) # save.npy

## 1차 모델 평가

## word2vec

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = okt_w2v
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("okt word2vec Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

okt word2vec Accuracy: 0.751836


### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("okt word2vec SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

okt word2vec SVC Accuracy: 0.790749


## Fasttext

### Logistic Regression

In [0]:
from sklearn.model_selection import train_test_split

X = okt_fxt
y = np.array(id_num)

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, 
                                                    random_state=RANDOM_SEED)

In [0]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

predicted = lgs.predict(X_test)

print("okt fasttext LR Accuracy: %f" % lgs.score(X_test, y_test))  #checking the accuracy

okt fasttext LR Accuracy: 0.824156


### SVC

In [0]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC(kernel='linear', C=1.0)
clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)
accuracy_score(y_test,y_pred)

print("okt fasttext SVC Accuracy: %f" % accuracy_score(y_test,y_pred))

okt fasttext SVC Accuracy: 0.849119
